## TMLDB와 MOVIE LENS 데이터 합치기

In [1]:
import pandas as pd
from IPython.display import clear_output
import math
import os
import time

### 1. 데이터 로드

- movies.csv : ML
- movies_metadata_en.xlsx : TM

In [2]:
ml_df = pd.read_csv("db/movies.csv")
tm_df = pd.read_excel("db/movies_metadata_en.xlsx")

C:\Users\dell\AppData\Local\Temp/ipykernel_27720/3668427069.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  tm_df = pd.read_excel("db/movies_metadata_en.xlsx")


### 2. 데이터 확인

In [3]:
tm_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
ml_df.loc[:10]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


- MOVIE LENS 데이터에 같은 영화 제목이 존재하는지 확인

In [5]:
trg_data = 'Kingdom Come'.lower().replace(' ', '')
for line in ml_df.iterrows():
    if trg_data in line[1]['title'].lower().replace(' ', ''):
        print(line[1]['title'])

Kingdom Come (2001)
Kingdom Come (2014)
WWE: Triple H: Thy Kingdom Come (2013)


In [6]:
test_tm_rd = str(tm_df.loc[0,'release_date'])[:4]
test_tm_rd

'2013'

In [7]:
len(ml_df)

58098

In [8]:
ml_df.loc[ml_df['title'].str.contains(test_tm_rd)]

,movieId,title,genres
19370,95595,Bela Kiss: Prologue (2013),Horror|Mystery|Thriller
20263,99007,Warm Bodies (2013),Comedy|Horror|Romance
20343,99335,Addicted (2013),Drama
20447,99721,Texas Chainsaw 3D (2013),Horror|Mystery|Thriller
20450,99728,Gangster Squad (2013),Action|Crime|Drama
...,...,...,...
57845,193289,The Optimists (2013),Documentary
57890,193403,A Talking Cat!?! (2013),Children|Comedy|Fantasy
57906,193457,Somewhere Else Tomorrow (2013),Documentary
57912,193469,The Happy Sad (2013),(no genres listed)


In [9]:
tm_df.loc[:5,['id','imdb_id', 'original_title', 'release_date']]

,id,imdb_id,original_title,release_date
0,141210,tt2250194,The Sleepover,2013-10-12
1,143750,tt2140519,The Farmer's Wife,2012-06-20
2,31610,tt0162711,Trixie,2000-06-28
3,29122,tt0218182,An Everlasting Piece,2000-12-22
4,68894,tt0238588,Shadow Magic,2000-09-08
5,136558,tt2043893,Kingdom Come,2011-01-01


### 3. 데이터 결합 알고리즘

In [10]:
# 완료된 데이터를 적재할 DataFrame
concated_data = pd.DataFrame(columns=['ml_id','tm_id','imdb_id','title_new','genres','release_date'])
if os.path.exists('db/concated_data.csv'):
    concated_data = pd.read_csv('db/concated_data.csv', index_col=0)
# 분류에 실패한 TM 데이터
failed_tm_data = pd.DataFrame(columns=['tm_id','title_new'])
if os.path.exists('db/failed_tm_data.csv'):
    failed_tm_data = pd.read_csv('db/failed_tm_data.csv', index_col=0)

- C: 완료된 수
- F: 실패한 수

In [12]:
# 분류!
start_time = time.time() # 처리 시간을 확인하기 위함입니다.
total_len = len(tm_df)
for line1 in tm_df.iterrows():
    if line1[1]['id'] in concated_data.loc[:,'tm_id'].values.tolist() or line1[1]['id'] in failed_tm_data.loc[:,'tm_id'].values.tolist():
        continue
    complete_switch = False
    temp_tm_title = str(line1[1]['original_title']).lower().replace(' ', '')
    temp_tm_rd = str(line1[1]['release_date'])[:4]
    for line2 in ml_df.loc[ml_df['title'].str.contains(temp_tm_rd)].iterrows():
        if temp_tm_title in line2[1]['title'].lower().replace(' ', ''):
            if temp_tm_rd in line2[1]['title'].lower().replace(' ', ''):
                concated_data.loc[len(concated_data),list(concated_data.columns)] = [line2[1]['movieId'],line1[1]['id'],line1[1]['imdb_id'],
                                                     line1[1]['original_title'],line2[1]['genres'],line1[1]['release_date']]
                complete_switch = True
                break
    if not complete_switch:
        failed_tm_data.loc[len(failed_tm_data),list(failed_tm_data.columns)] = [line1[1]['id'],line1[1]['original_title']]
    
    # 5개마다 저장
    if len(concated_data) != 0 and len(concated_data) % 5 == 0:
        concated_data.to_csv('db/concated_data.csv', index=True)
    
    if len(failed_tm_data) != 0 and len(failed_tm_data) % 5 == 0:
        failed_tm_data.to_csv('db/failed_tm_data.csv', index=True)
        
    # 작업이 100개 완료될 때마다 정보 출력
    if (line1[0]+1) % 100 == 0:
        clear_output(wait = True)
        blockCount = int(math.floor(((line1[0]+1)/total_len)*20))
        dur_time = time.time()-start_time
        dur_expected_time = dur_time*(total_len/line1[0]+1)
        print("Preprocessing!")
        print('PROGRESS - [{:d}/{:d}] [C:{:d}, F:{:d}] [{:}] {:.0%}'.format(line1[0]+1, total_len, len(concated_data), len(failed_tm_data), '#' * blockCount + '-' * (20 - blockCount), (line1[0]+1)/total_len))
        print('Processing time: {:.1f}s / {:.0f}s [{:.0f}s]'.format(dur_time, dur_expected_time, dur_expected_time-dur_time))
concated_data.to_csv('db/concated_data.csv', index=True)
failed_tm_data.to_csv('db/failed_tm_data.csv', index=True)
end_time = time.time()
clear_output(wait = True)
print("Complete!")
print('COMPLETE - [{:d}/{:d}] [C:{:d}, F:{:d}] [{:}] 100%'.format(total_len, total_len, len(concated_data), len(failed_tm_data), '#' * 20))
print('processing time: {:.1f}s'.format(end_time-start_time))

Complete!
COMPLETE - [9939/9939] [C:7469, F:2455] [####################] 100%
processing time: 663.9s


### 4. 성공 데이터 확인

In [16]:
concated_data

,ml_id,tm_id,imdb_id,title_new,genres,release_date
0,3721,31610,tt0162711,Trixie,Comedy|Crime|Mystery,2000-06-28 00:00:00
1,4245,68894,tt0238588,Shadow Magic,Drama|Romance,2000-09-08 00:00:00
2,4742,248757,tt0160710,Punks,Comedy,2000-01-24 00:00:00
3,4891,140511,tt0177888,King of the Jungle,Drama|Thriller,2000-10-12 00:00:00
4,4991,292917,tt0274868,Some Body,Comedy|Drama|Romance,2001-01-19 00:00:00
...,...,...,...,...,...,...
7464,103454,155,tt0468569,The Dark Knight,Documentary,2008-07-16 00:00:00
7465,79132,27205,tt1375666,Inception,Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,2010-07-14 00:00:00
7466,122912,299536,tt4154756,Avengers: Infinity War,Action|Adventure|Sci-Fi,2018-04-25 00:00:00
7467,89104,18634,tt0450367,Puzzlehead,Drama|Sci-Fi,2005-04-21 00:00:00


### 5. 실패 데이터 확인

In [14]:
failed_tm_data

,tm_id,title_new
0,141210,The Sleepover
1,143750,The Farmer's Wife
2,29122,An Everlasting Piece
3,136558,Kingdom Come
4,55479,Pi챰ero
...,...,...
2450,429617,Spider-Man: Far From Home
2451,335983,Venom
2452,24428,The Avengers
2453,475557,Joker
